In [1]:
import os
import datetime
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from joblib import dump, load

### 导入相关的数据集

#### 室外温度湿度数据输入

In [2]:
# Outdoor_TH_path = "data/outdoor_HT.csv"

def outdoor_TH_Import(path):
    Outdoor_TH = pd.read_csv(path, encoding='utf-8')
    
    values=[]
    for i in Outdoor_TH.values:
        values.append(i[0].split(";"))

    columns_name=[]
    for i in Outdoor_TH.columns[0].split(';'):
        columns_name.append(i.replace('"',"").replace(" ",""))

    df_outdoor_TH = pd.DataFrame(values, columns=columns_name)
    df_outdoor_TH.rename(columns={'室外温度Time':'时间'}, inplace = True)

    df_outdoor_TH = df_outdoor_TH.drop(['室外湿度Time'],axis=1)

    df_outdoor_TH['时间'] = pd.to_datetime(df_outdoor_TH['时间'])
    df_outdoor_TH[df_outdoor_TH.columns[1:]] = df_outdoor_TH[df_outdoor_TH.columns[1:]].astype(float)
    
    data_outdoor_two_minutes =[]
    for i in range(len(df_outdoor_TH)):
        for j in range(9):
            data_outdoor_two_minutes.append(df_outdoor_TH.loc[i,:] + (j+1)*(df_outdoor_TH.diff()/10)[1:].loc[j+1,:])

    df_outdoor_TH = df_outdoor_TH.append(data_outdoor_two_minutes)
    df_outdoor_TH = df_outdoor_TH.sort_values(by="时间").reset_index(drop=True)
    df_outdoor_TH = df_outdoor_TH.set_index('时间')
    
    return df_outdoor_TH

#### 外部风机电流数据输入

In [3]:
# fan_elec_path = "data/fan_elec.csv"

def fan_elec_Import(path):
    fan_elec = pd.read_csv(path, encoding='utf-8')

    values=[]
    for i in fan_elec.values:
        values.append(i[0].split(";"))

    columns_name=[]
    for i in fan_elec.columns[0].split(';'):
        columns_name.append(i.replace('"',"").replace(" ",""))

    df_fan_elec = pd.DataFrame(values, columns=columns_name)
    df_fan_elec.rename(columns={'KZS5送风机电流Time':'时间'}, inplace = True)

    df_fan_elec['时间'] = pd.to_datetime(df_fan_elec['时间'])
    df_fan_elec=df_fan_elec.set_index('时间')
    
    return df_fan_elec

# def fan_elec_import_concat(path1, path2):
#     fan_elec_import1 = fan_elec_Import(path1)
#     fan_elec_import2 = fan_elec_Import(path2)[fan_elec_Import(path2).columns[[0,1]]]

#     fan_elec = pd.concat([fan_elec_import1, fan_elec_import2], axis=0, ignore_index = True)
    
#     return fan_elec

#### 回风温度数据输入

In [4]:
# Huifeng_T_path = "data/huifeng_T.csv"

def Huifeng_T_Import(path):
    huifeng_T = pd.read_csv("data/huifeng_T.csv", encoding='utf-8')

    values=[]
    for i in huifeng_T.values:
        values.append(i[0].split(";"))

    columns_name=[]
    for i in huifeng_T.columns[0].split(';'):
        columns_name.append(i.replace('"',"").replace(" ",""))

    df_huifeng_T = pd.DataFrame(values, columns=columns_name)
    df_huifeng_T.rename(columns={'回风温度Time':'时间'}, inplace = True)

    df_huifeng_T['时间'] = pd.to_datetime(df_huifeng_T['时间'])

    df_huifeng_T=df_huifeng_T.set_index('时间')
    
    return df_huifeng_T

#### 表冷阀开度反馈输入

In [5]:
# biaoleng_path = "data/biaoleng.csv"

def biaoleng_Import(path):
    biaoleng = pd.read_csv(path, encoding='utf-8')

    values=[]
    for i in biaoleng.values:
        values.append(i[0].split(";"))

    columns_name=[]
    for i in biaoleng.columns[0].split(';'):
        columns_name.append(i.replace('"',"").replace(" ",""))

    biaoleng_T = pd.DataFrame(values, columns=columns_name)
    biaoleng_T.rename(columns={'表冷阀开度反馈Time':'时间'}, inplace = True)

    biaoleng_T['时间'] = pd.to_datetime(biaoleng_T['时间'])

    biaoleng_T=biaoleng_T.set_index('时间')

    return biaoleng_T

#### 测量数据输入

In [6]:
def measure_data_Import():

    path = os.getcwd()
    files = os.listdir(path+"\data\Odata")
    files_csv = [f for f in files if f[-3:] == 'csv']

    df = pd.DataFrame()
    for f in files_csv:
        data = pd.read_csv('data/Odata/'+ f, encoding='utf-8')
        df = df.append(data)

    values=[]
    for i in df.values:
        values.append(i[0].split(";"))

    columns_name=[]
    for i in df.columns[0].split(';'):
        columns_name.append(i.replace('"',"").replace(" ",""))

    df_data = pd.DataFrame(values, columns=columns_name)
    df_data.rename(columns={'KZS5\加热蒸汽阀位反馈Time':'时间'}, inplace = True)

    df_data['时间'] = pd.to_datetime(df_data['时间'])

    df_data=df_data.set_index('时间')
    df_data=df_data.drop(['表冷阀开度反馈Time'],axis=1)
    
    #给到的表冷阀的数据是错误的，需要单独引入。Drop掉当前的表冷阀开度反馈列
    df_data = df_data.drop(df_data.filter(regex='表冷阀开度').columns, axis=1)
    
    return df_data

 #### 加上回风温度数据,外部风机电流数据,外部温度湿度数据

In [7]:
def df_data_concat(biaoleng_path,huifeng_T_path,fan_elec_path,outdoor_TH_path):   
    
    df_measure_data = measure_data_Import()
    df_fan_elec = fan_elec_Import(fan_elec_path)
    df_outdoor_TH = outdoor_TH_Import(outdoor_TH_path)
    df_huifeng_T = Huifeng_T_Import(huifeng_T_path)
    df_biaoleng = biaoleng_Import(biaoleng_path)
    
    df_data = pd.concat([df_measure_data, df_fan_elec, df_outdoor_TH, df_huifeng_T, df_biaoleng], axis=1, join='inner')

    df_data.drop([col for col in df_data.columns if 'Time' in col],axis=1,inplace=True)
    df_data.drop(['风机运行ValueY'],axis=1,inplace=True)

    # 移除掉 ValueY
    for i in df_data.columns[0:]:
        df_data.rename(columns={i:i[:-6]}, inplace = True)

    for i in df_data.columns[df_data.columns.str.contains('KZS5')]:
        df_data.rename(columns={i:i[5:]}, inplace = True)

    df_data.rename(columns={'送风湿度1': '送风湿度'}, inplace = True)
    df_data.rename(columns={'风机电流': '送风机电流'}, inplace = True)
    
    df_data.drop(['室外温度', '室外湿度'],axis=1, inplace=True)

    df_data = df_data.drop_duplicates()
    df_data = df_data.astype(float)

    return df_data

#### 天气预报数据导入

In [8]:
def Outdoor_weather_Append():
    
    path = os.getcwd()
    files = os.listdir(path+"\data\weather")
    files_csv = [f for f in files if f[-3:] == 'csv']

    outdoor_weather = pd.DataFrame()
    for f in files_csv:
        data = pd.read_csv('data/weather/'+ f, encoding='utf-8')
        outdoor_weather = outdoor_weather.append(data)
        
#     outdoor_import1 = Outdoor_weather_Import(path1)
#     outdoor_import2 = Outdoor_weather_Import(path2)
                           
    outdoor_weather.drop(outdoor_weather.columns[-6:], inplace = True, axis = 1)
    outdoor_weather['风向/度'] = outdoor_weather['风向/度'].astype(str)

    values =[]
    for i in outdoor_weather['风向/度']:
        values.append(i.split("/")[0])
    outdoor_weather['风向/度'] = values

    outdoor_weather['时间'] = pd.to_datetime(outdoor_weather['时间'])
    outdoor_weather_selected = outdoor_weather[outdoor_weather.columns[[0,1,3]]]

    outdoor_weather_selected=outdoor_weather_selected.set_index('时间')
    
    return outdoor_weather_selected

In [9]:
# 天气预报数据从小时到分钟的分割
def Outdoor_weather_transform_mins(mins=2):
    
    outdoor_weather_append = Outdoor_weather_Append().reset_index()
    outdoor_weather_append_diff = (outdoor_weather_append.diff()/(60/mins))[1:].reset_index(drop=True)

    #Now, append the 
    df_add_append=pd.DataFrame()
    outdoor_weather_append_add = outdoor_weather_append[:-1].reset_index(drop=True)

    steps = int((60/mins))-1

    for i in range(steps):
        outdoor_weather_append_add = outdoor_weather_append_add + outdoor_weather_append_diff
        df_add_append = df_add_append.append(outdoor_weather_append_add, ignore_index=True)

    outdoor_weather_append = outdoor_weather_append.append(df_add_append,ignore_index=True)    
    outdoor_weather_append = outdoor_weather_append.sort_values(by=['时间']).reset_index(drop=True)
    
    return outdoor_weather_append

### 5/28-7/7 数据探索

#### 引入5/28以后的数据

In [10]:
def measure_data_Import_528(fan_elec_path_528, data_528_path):

    df_528_dianliu = fan_elec_Import(fan_elec_path_528)
    df_528_dianliu.drop(df_528_dianliu.columns[1:], axis=1, inplace=True)

    df_528 = pd.read_csv(data_528_path, encoding='utf-8')
    
    values=[]
    for i in df_528.values:
        values.append(i[0].split(";"))

    columns_name=[]
    for i in df_528.columns[0].split(';'):
        columns_name.append(i.replace('"',"").replace(" ",""))

    df_data_528 = pd.DataFrame(values, columns=columns_name)
    df_data_528.rename(columns={'KZS5\加热蒸汽阀调节输出Time':'时间'}, inplace=True)

    df_data_528[df_data_528.columns[0]]= pd.to_datetime(df_data_528[df_data_528.columns[0]])
    df_data_528.set_index(df_data_528.columns[0],inplace=True)
    df_data_528.drop([col for col in df_data_528.columns if 'Time' in col],axis=1,inplace=True)
    
    # append 风机电流数据
    df_data_528 = pd.concat([df_data_528,df_528_dianliu],axis=1,join='inner')
    
    # 移除掉 ValueY
    for i in df_data_528.columns:
        df_data_528.rename(columns={i:i[:-6]}, inplace = True)

    for i in df_data_528.columns[df_data_528.columns.str.contains('KZS5')]:
        df_data_528.rename(columns={i:i[5:]}, inplace = True)

    df_data_528 = df_data_528.drop_duplicates()
    df_data_528.rename(columns={'送风湿度1':'送风湿度'}, inplace= True)
    df_data_528.rename(columns={'风机电流': '送风机电流'}, inplace = True)
    
    return df_data_528

In [11]:
def df_data_528_selected(fan_elec_path_528,data_528_path,biaoleng_path,huifeng_T_path,fan_elec_path,outdoor_TH_path):
    
    df_data_528 = measure_data_Import_528(fan_elec_path_528, data_528_path)
    
    global df_concat
    df_concat = df_data_concat(biaoleng_path,huifeng_T_path,fan_elec_path,outdoor_TH_path)
    
    # 与前三个月比较，相同纬度的数据
    df_data_528_No_added = df_data_528[df_data_528.columns[df_data_528.columns.isin(df_concat.columns)]]
    
    # 与前三个月的数据比较，增加维度的数据
    df_data_528_added = df_data_528[df_data_528.columns[~df_data_528.columns.isin(df_concat.columns)]]
    
    # 对于这些增加的维度数据，我们可以看到有些维度的数据很多都是sparse value.
    df_data_528_added.drop([df_data_528_added.columns[4]],axis=1, inplace=True)
    
    df_data_528_added  = df_data_528_added.astype(float)
#     df_data_528_added.describe().to_csv('528_new.csv')

    df_data_528_added_selected = pd.concat([df_data_528_added[['表冷器进水温度','表冷器出水温度']],df_data_528_No_added],axis=1)
    df_data_528_added_selected = df_data_528_added_selected.astype(float)
    
    return df_data_528_No_added, df_data_528_added, df_data_528_added_selected

### 重新整合空调测量数据

#### 训练数据1 - Append 5月28日以后的数据到之前3个月的数据上  (会缺少新加的数据维度)。
#### 训练数据2 - 只能用5月28日以后的数据 - 因为多了两个维度[`表冷器进水温度`,`表冷器出水温度`]

In [12]:
def data_select_append(fan_elec_path_528,data_528_path,biaoleng_path,huifeng_T_path,fan_elec_path,outdoor_TH_path):
    #确定选择出来的维度与空调开关机模型的维度一致
    df_data_528_No_added, df_data_528_added, df_data_528_added_selected = df_data_528_selected(fan_elec_path_528,data_528_path,biaoleng_path,huifeng_T_path,fan_elec_path,outdoor_TH_path)
    # set(df_concat.columns) == set(df_data_528_No_added.columns)
    df_concat_528added = df_concat.append(df_data_528_No_added)
    
#     new_col = ['TAT7205-1T', 'TAT7205-1H', 'TAT7205-2H', 'TAT7205-2T', 'TAT7205-3H',
#                'TAT7205-3T', 'TAT7205-4H', 'TAT7205-4T', 'TAT7205-5H', 'TAT7205-5T',
#                '加湿蒸汽阀位反馈', '加热蒸汽阀位反馈', '回风温度', '回风湿度', '新风温度', '新风湿度', '新风阀门反馈',
#                '混风阀门反馈', '表冷阀开度反馈', '送风机电流', '送风温度', '送风湿度']

#     df_concat_528added = df_concat_528added[new_col]
    
    print('5月28日后增加的数据维度: {} '.format(df_data_528_added.columns))
    
    print('训练数据1 - 不考虑5月28日后增加的数据维度: {} '.format(df_concat_528added.columns))
    print('训练数据2 - 基于5月28日数据后的数据维度: {} '.format(df_data_528_added_selected.columns))
    
    return df_concat_528added, df_data_528_added_selected

#### 引入MES物料数据

In [13]:
def wuliao_mes_import_transform(wuliao_mes_path, wuliao_mes_pivot_path):

    # wuliao_mes = pd.read_csv('data/wuliao/mergemes_01.csv',encoding='utf-8')
    wuliao_mes = pd.read_csv(wuliao_mes_path,encoding='utf-8')

    wuliao_mes['物料时间'] = pd.to_datetime(wuliao_mes['datetime'])
    wuliao_mes.drop(['datetime'],axis=1,inplace=True)

    wuliao_mes.set_index('物料时间',inplace=True)

    # 由于MES里3k的加料数据，是针对预配柜的加料，所以有很强的滞后性，会拿掉在MES里3K产线上的加料数据。
    mask_not_selected_3k = (wuliao_mes.filter(like = 'T_LAP3').columns).append(wuliao_mes.filter(like = 'T_LAC3').columns)
    wuliao_mes_59 = wuliao_mes[wuliao_mes.columns[~wuliao_mes.columns.isin(mask_not_selected_3k)]]
    
    #引入MES物料数据的pivot table
#     wuliao_mes_pivot = pd.read_csv('data/wuliao/wuliao_mes_pivot.csv',header=None,encoding='utf-8')
    wuliao_mes_pivot = pd.read_csv(wuliao_mes_pivot_path,header=None,encoding='utf-8')

    wuliao_mes_pivot.drop([2],axis=1,inplace=True)
    wuliao_mes_pivot.dropna(inplace=True)

    wuliao_mes_59_col_name=[]
    for i in range(len(wuliao_mes_59.columns)):
         wuliao_mes_59_col_name.append(wuliao_mes_pivot[wuliao_mes_pivot[1] == wuliao_mes_59.columns[i]][0].values[0])

    wuliao_mes_59.columns = wuliao_mes_59_col_name
    
    print('拿掉MES里3K产线上的加料数据后的物料数据维度: {} '.format(wuliao_mes_59.columns))
    
    return wuliao_mes_59

#### 引入3k物料加料的流量、温湿度数据 和 3/5/9k产线出口温度

In [14]:
# 对引入的物料数据做预处理
def convert(args):
    return pd.Timedelta(hours=int(args[:-3]),minutes=int(args[-2:]))

def wuliao_3k_jiaoliao_import_transform_concat(wuliao_mes_59, \
                                                wuliao_3k_jialiao_liuliang_path, \
                                                wuliao_3k_jialiao_HT_path, \
                                                wuliao_3k_chuyegui_t_path, \
                                                wuliao_5k_chuyegui_t_path, \
                                                wuliao_9k_chuyegui_t_path):
    
    """
    在处理过的MES数据表增加 '三千加料烟叶流量实际值' 数据维度：
    
    """ 
    wuliao_mes_3k_mass = pd.read_csv(wuliao_3k_jialiao_liuliang_path, encoding='utf-8')

    # wuliao_mes_3k_mass[['进柜日期','出柜日期']] = wuliao_mes_3k_mass[['进柜日期','出柜日期']].astype('datetime64[ns]')
    wuliao_mes_3k_mass[['进柜日期']] = wuliao_mes_3k_mass[['进柜日期']].astype('datetime64[ns]')

    # timedelta_col = ['进柜开始时间','进柜结束时间','出柜开始时间','出柜结束时间']
    timedelta_col = ['进柜开始时间','进柜结束时间']

    for i in range(len(timedelta_col)):
        wuliao_mes_3k_mass[timedelta_col[i]] = wuliao_mes_3k_mass[timedelta_col][timedelta_col[i]].apply(convert)

    wuliao_mes_3k_mass['产量kg'] = wuliao_mes_3k_mass['产量kg'].astype(int)

    wuliao_mes_3k_mass['进柜时间diff'] = wuliao_mes_3k_mass['进柜结束时间']- wuliao_mes_3k_mass['进柜开始时间']
    # wuliao_mes_3k_mass['出柜时间diff'] = wuliao_mes_3k_mass['出柜结束时间']- wuliao_mes_3k_mass['出柜开始时间']

    wuliao_mes_3k_mass['进柜开始时间'] = wuliao_mes_3k_mass['进柜日期'] + wuliao_mes_3k_mass['进柜开始时间']
    # wuliao_mes_3k_mass['出柜开始时间'] = wuliao_mes_3k_mass['出柜日期'] + wuliao_mes_3k_mass['出柜开始时间']

    wuliao_mes_3k_mass['进柜结束时间'] = (wuliao_mes_3k_mass[wuliao_mes_3k_mass['进柜时间diff'] >= pd.Timedelta(minutes=0)]['进柜日期'] + wuliao_mes_3k_mass[wuliao_mes_3k_mass['进柜时间diff'] >= pd.Timedelta(minutes=0)]['进柜结束时间']).append(
    (wuliao_mes_3k_mass[wuliao_mes_3k_mass['进柜时间diff'] < pd.Timedelta(minutes=0)]['进柜日期'] + wuliao_mes_3k_mass[wuliao_mes_3k_mass['进柜时间diff'] < pd.Timedelta(minutes=0)]['进柜结束时间']+pd.Timedelta(days=1)))

    # wuliao_mes_3k_mass['出柜结束时间'] = (wuliao_mes_3k_mass[wuliao_mes_3k_mass['出柜时间diff'] >= pd.Timedelta(minutes=0)]['出柜日期'] + wuliao_mes_3k_mass[wuliao_mes_3k_mass['出柜时间diff'] >= pd.Timedelta(minutes=0)]['出柜结束时间']).append(
    # (wuliao_mes_3k_mass[wuliao_mes_3k_mass['出柜时间diff'] < pd.Timedelta(minutes=0)]['出柜日期'] + wuliao_mes_3k_mass[wuliao_mes_3k_mass['出柜时间diff'] < pd.Timedelta(minutes=0)]['出柜结束时间']+pd.Timedelta(days=1)))

    # wuliao_mes_3k_mass.drop(['进柜日期','出柜日期','进柜时间diff','出柜时间diff'],axis=1,inplace=True)
    wuliao_mes_3k_mass.drop(['进柜日期','进柜时间diff'],axis=1,inplace=True)

    wuliao_mes_3k_mass['进柜流量'] = wuliao_mes_3k_mass['产量kg']/((wuliao_mes_3k_mass['进柜结束时间'] - wuliao_mes_3k_mass['进柜开始时间']).dt.total_seconds()/60)
    # wuliao_mes_3k_mass['出柜流量'] = wuliao_mes_3k_mass['产量kg']/((wuliao_mes_3k_mass['出柜结束时间'] - wuliao_mes_3k_mass['出柜开始时间']).dt.total_seconds()/60)
    
    ### 增加 '三千加料烟叶流量实际值' 数据维度
    wuliao_mes_59['三千加料烟叶流量实际值'] = 0.00
    # wuliao_mes_59['三千烘丝入口烟叶流量'] = 0.00

    for j in range(len(wuliao_mes_3k_mass)):
        mask_jingui = (wuliao_mes_59.index >= wuliao_mes_3k_mass['进柜开始时间'][j]) & (wuliao_mes_59.index <= wuliao_mes_3k_mass['进柜结束时间'][j])
    #     mask_chugui = (wuliao_mes_59.index >= wuliao_mes_3k_mass['出柜开始时间'][j]) & (wuliao_mes_59.index <= wuliao_mes_3k_mass['出柜开始时间'][j])

        wuliao_mes_59['三千加料烟叶流量实际值'][mask_jingui] = wuliao_mes_3k_mass['进柜流量'][j]
    #     wuliao_mes_59['三千烘丝入口烟叶流量'][mask_chugui] = wuliao_mes_3k_mass['出柜流量'][j]
    
    
    """
    在处理过的MES数据表增加 '三千加料出口温度实际值' 数据维度：
    
    """ 
    
    ### 增加 '三千加料出口温度实际值' 数据维度：
    wuliao_mes_3k_HT = pd.read_csv(wuliao_3k_jialiao_HT_path, encoding='utf-8')
    
    wuliao_mes_3k_HT = wuliao_mes_3k_HT[['牌号批次','测量时间', '温度1', '温度2', '温度3']].dropna()

    jinyegui_3k_min = wuliao_mes_3k_HT[['温度1','温度2','温度3']].min().min()
    jinyegui_3k_max = wuliao_mes_3k_HT[['温度1','温度2','温度3']].max().max()

    wuliao_mes_59['三千加料出口温度实际值'] = 0.0

    # 按照业务逻辑，当流量>0时，则表示温度测量点会有物料
    mask_3k_jinyegui= (wuliao_mes_59['三千加料烟叶流量实际值'] > 0)
    wuliao_mes_59['三千加料出口温度实际值'][mask_3k_jinyegui] = np.random.uniform(jinyegui_3k_min,jinyegui_3k_max,len(mask_3k_jinyegui.loc[mask_3k_jinyegui==True]))
    
    
    """
    3/5/9k产线出口温度 - 引入各产线的出口温度测量值并生成各个测量实际点的温度值
    
    """
    chuyegui_3k = pd.read_csv(wuliao_3k_chuyegui_t_path)
    chuyegui_5k = pd.read_csv(wuliao_5k_chuyegui_t_path)
    chuyegui_9k = pd.read_csv(wuliao_9k_chuyegui_t_path)

    # 添加各产线的出口物料温度
    chuyegui_3k_min = chuyegui_3k[['温度1','温度2','温度3']].min().min()
    chuyegui_3k_max = chuyegui_3k[['温度1','温度2','温度3']].max().max()

    chuyegui_5k_min = chuyegui_5k[['温度1','温度2','温度3']].min().min()
    chuyegui_5k_max = chuyegui_5k[['温度1','温度2','温度3']].max().max()

    chuyegui_9k_min = chuyegui_9k[['温度1','温度2','温度3']].min().min()
    chuyegui_9k_max = chuyegui_9k[['温度1','温度2','温度3']].max().max()

    wuliao_mes_59['三千烘丝烟叶温度随机值'] = 0.0
    wuliao_mes_59['五千烘丝烟叶温度随机值'] = 0.0
    wuliao_mes_59['九千烘丝烟叶温度随机值'] = 0.0

    ### 按照业务逻辑，当流量>0时，则表示温度测量点会有物料
    mask_3k_chugui= (wuliao_mes_59['三千烘丝入口烟叶流量'] > 0)
    mask_5k_chugui= (wuliao_mes_59['五千烘丝入口烟叶流量'] > 0)
    mask_9k_chugui= (wuliao_mes_59['九千RCC入口烟叶流量'] > 0)

    wuliao_mes_59['三千烘丝烟叶温度随机值'][mask_3k_chugui] = np.random.uniform(chuyegui_3k_min,chuyegui_3k_max,len(mask_3k_chugui.loc[mask_3k_chugui==True]))
    wuliao_mes_59['五千烘丝烟叶温度随机值'][mask_5k_chugui] = np.random.uniform(chuyegui_5k_min,chuyegui_5k_max,len(mask_5k_chugui.loc[mask_5k_chugui==True]))
    wuliao_mes_59['九千烘丝烟叶温度随机值'][mask_9k_chugui] = np.random.uniform(chuyegui_9k_min,chuyegui_9k_max,len(mask_9k_chugui.loc[mask_9k_chugui==True]))

    return wuliao_mes_59

### 合并以上所有数据
#### 空调数据(训练数据1)、合并物料和天气预报数据

In [15]:
def df_final_concat_01(df_concat_528added,wuliao_mes_59_final,outdoor_weather_append):
    # 发现在时间戳3-18下午1点到2点的时间里，时间戳的格式发生了变化。所以，对之前的和之后的时间戳的预处理会不一样。
    a = df_concat_528added[df_concat_528added.index > pd.Timestamp('2020-03-18 14:10:16')].index.floor('min')
    b = df_concat_528added[df_concat_528added.index < pd.Timestamp('2020-03-18 14:10:16')].index.floor('min') + pd.Timedelta(minutes=1)

    df_concat_528added['物料时间'] = (b.append(a)).to_series().values

    df_concat_528_wuliao_added = df_concat_528added.reset_index().merge(wuliao_mes_59_final.reset_index(), how="left", on="物料时间")
    df_concat_528_wuliao_added.set_index('时间',inplace=True)

    # df_concat_528_wuliao_added.drop(['物料时间'],axis=1,inplace=True)
    # df_concat_528_wuliao_added = df_concat_528_wuliao_added.astype(float)

    df_concat_528_wuliao_added.dropna(inplace=True)

    #### 合并天气预报数据
    df_concat_528_wuliao_added['天气预报时间'] = df_concat_528_wuliao_added['物料时间']

    outdoor_weather_append.rename(columns={'时间':'天气预报时间', '温度℃':'天气预报温度','相对湿度%':'天气预报湿度'}, inplace=True)
    df_concat_weather = df_concat_528_wuliao_added.reset_index().merge(outdoor_weather_append, how="left", on="天气预报时间")
    df_concat_weather.drop(['天气预报时间','物料时间'], axis=1, inplace=True)
    df_concat_weather.dropna(inplace=True)

    df_concat_weather = df_concat_weather.set_index('时间')
    df_concat_weather = df_concat_weather.astype(float)

    return df_concat_weather

In [17]:
if __name__=="__main__":
    
    start =time.clock()
    
    # 定义IoT点位CSV数据路径
    fan_elec_path_528 = "data/fan_elec_528.csv"
    data_528_path = 'data/Odata/528/528_0707.csv'
    biaoleng_path = "data/biaoleng.csv"
    huifeng_T_path = "data/huifeng_T.csv"
    fan_elec_path = "data/fan_elec.csv"
    outdoor_TH_path = "data/outdoor_HT.csv"
    
    # 定义MES物料数据、3k加料流量和温湿度、各产线储叶柜温度CSV数据路径
    wuliao_mes_path = 'data/wuliao/mergemes_01.csv'
    wuliao_mes_pivot_path = 'data/wuliao/wuliao_mes_pivot.csv'
    
    wuliao_3k_jialiao_liuliang_path = 'data/wuliao/3k/3kwuliao_01.csv'
    wuliao_3k_jialiao_HT_path = 'data/wuliao/3k/3k_wuliao_in_HT.csv'

    wuliao_3k_chuyegui_t_path = 'data\wuliao\chuye_T\chuyegui_3k_t.csv'
    wuliao_5k_chuyegui_t_path = 'data\wuliao\chuye_T\chuyegui_5k_t.csv'
    wuliao_9k_chuyegui_t_path = 'data\wuliao\chuye_T\chuyegui_9k_t.csv'    

    # outdoor_weather_path = "data/outdoor_weather.csv"
    # outdoor_weather_path_5 = "data/outdoor_weather_5.csv"
    # jialiao_path = "data\product_plan\jialiao_order0529.csv"

    # 引入空调测量数据
    df_concat_528added, df_data_528_added_selected = data_select_append(fan_elec_path_528,
                                                                        data_528_path,
                                                                        biaoleng_path,
                                                                        huifeng_T_path,
                                                                        fan_elec_path,
                                                                        outdoor_TH_path)
    print('空调测量数据已经成功引入')
    
    # 引入物料数据
    wuliao_mes_59 = wuliao_mes_import_transform(wuliao_mes_path, wuliao_mes_pivot_path)
    wuliao_mes_59_final = wuliao_3k_jiaoliao_import_transform_concat(wuliao_mes_59, 
                                                                    wuliao_3k_jialiao_liuliang_path, 
                                                                    wuliao_3k_jialiao_HT_path, 
                                                                    wuliao_3k_chuyegui_t_path, 
                                                                    wuliao_5k_chuyegui_t_path, 
                                                                    wuliao_9k_chuyegui_t_path)
   
    print('物料数据已经成功引入')
    
    # 引入天气预报数据
    outdoor_weather_append = Outdoor_weather_transform_mins(mins=2)    
    print('天气预报数据已经成功引入')

    # 合并数据
    df_concat_weather = df_final_concat_01(df_concat_528added,wuliao_mes_59_final,outdoor_weather_append)
    print('所有数据已经成功引入和合并')

    # 保存预处理的数据到本地csv，供训练数据使用
    today_string = datetime.datetime.today().strftime('%Y_%m_%d')
    df_concat_weather.to_csv('wentai_data_pre/wentai_data_' + today_string + '.csv',index=True)
    
    print('最终训练数据1的字段: {} '.format(df_concat_weather.columns))
    
    print('预处理的数据已经保存到本地csv')
    
    end = time.clock()
    print('总共预处理时间: %s Seconds'%(end-start))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


5月28日后增加的数据维度: Index(['加热蒸汽阀调节输出', '加湿蒸汽阀调节输出', '加权平均湿度', '加权平均温度', '蒸汽压力', '表冷进水压力',
       '表冷器进水温度', '表冷器出水温度', '表冷阀开度设定', 'FT7205-1加热蒸汽流量瞬时值',
       'FQ7205-1加热蒸汽流量累积值', 'FT7205-2加湿蒸汽流量瞬时值', 'FQ7205-2加湿蒸汽流量累积值',
       'FT7205-3冷冻水流量瞬时值', 'FQ7205-3冷冻水流量累积值'],
      dtype='object') 
训练数据1 - 不考虑5月28日后增加的数据维度: Index(['TAT7205-1H', 'TAT7205-1T', 'TAT7205-2H', 'TAT7205-2T', 'TAT7205-3H',
       'TAT7205-3T', 'TAT7205-4H', 'TAT7205-4T', 'TAT7205-5H', 'TAT7205-5T',
       '加湿蒸汽阀位反馈', '加热蒸汽阀位反馈', '回风温度', '回风湿度', '新风温度', '新风湿度', '新风阀门反馈',
       '混风阀门反馈', '表冷阀开度反馈', '送风机电流', '送风温度', '送风湿度'],
      dtype='object') 
训练数据2 - 基于5月28日数据后的数据维度: Index(['表冷器进水温度', '表冷器出水温度', '加热蒸汽阀位反馈', '加湿蒸汽阀位反馈', '送风湿度', '送风温度',
       'TAT7205-1T', 'TAT7205-1H', 'TAT7205-2T', 'TAT7205-2H', 'TAT7205-3T',
       'TAT7205-3H', 'TAT7205-4T', 'TAT7205-4H', 'TAT7205-5T', 'TAT7205-5H',
       '回风温度', '回风湿度', '新风温度', '新风湿度', '表冷阀开度反馈', '新风阀门反馈', '混风阀门反馈', '送风机电流'],
      dtype='object') 
空调测量数据已经成功引入
拿掉MES里3K产线上的加料数据后的

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:64: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
